In [71]:
'''
This Notebook demonstrates the use of cloudFoundary services
from GE Predix.  Specifically it uses UAA authentication
GE Asset microservices for a simple web-based application.

The python-based script uses:
    subprocess management (standard library)
    json (standard library)
    cloudFoundry api, linux curl
    cloudFoundry cf CLI
    cloufFoundary UAA and GE Asset micrososervices

References:
The process is based on the example at
    https://www.predix.io/blog/article.html?article_id=2034
with additional information from 
    http://stackoverflow.com/questions/27985469/how-to-get-oauth-token-from-cloudfoundry
    https://docs.python.org/3.7/reference/index.html
    https://docs.python.org/3.7/library/index.html
'''

x=1

In [72]:
'''
Procedure:

Login
1> mkdir ~/tempForBlog1; cd ~/tempForBlog1
2> cf login # with your credentials

Create UAA Service
3> cf create-service predix-uaa Tiered my-uaa -c '{"adminClientSecret":"my-secret"}'

Get UAA Vitals
4> appname=`whoami`.temp.html # need unique name
5> touch $appname
6> cf push --no-start $appname
7> cf bs $appname my-uaa 
8> cf env $appname # Output is environment variables
9> cf delete $appname --f

Create Asset Service
10> cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

Get Asset Vitals
11> touch $appname
12> cf push --no-start $appname
13> cf bs $appname my-asset
14> cf env $appname

UAAC Targets UAA
15> uaac target https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io
16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset

Add UAAC Client
17> uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --secret my-secret --name asset-client






cf curl /v2/spaces
cf curl /v2/apps
cf curl /v2/services



As suggested in the reference - we ran an dummy app (cwinsor.temp.html) as a means to get the "issuerId"
We want to reproduce the process using cf curl.  This involves tracing JSON urls to find it.
The final target is:
cf env cwinsor.temp.html ...
 "issuerId": "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

To reproduce that using cf curl
cf curl /v2/spaces
produces
["resources"][0]["metadata"]["url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc",
["resources"][0]["entity"]["name"] = "dev"
["resources"][0]["entity"]["apps_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
["resources"][0]["entity"]["service_instances_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances",

<first find the UAA service GUID>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
produces
["resources"][0]["entity"]["name"]: "my-uaa"
["resources"][0]["metadata"]["guid"]: "5b5428fa-4922-4be6-81ed-ad40a2a28f70",

<now that we have the UAA GUID - search for that in my app as a service...>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
produces
["resources"][0]["entity"]["name"]: "cwinsor.temp.html"
["resources"][0]["metadata"]["url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d",
["resources"][0]["entity"]["service_bindings_url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings",

cf curl /v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings
produces
["resources"][0]["entity"][service_instance_guid": "5b5428fa-4922-4be6-81ed-ad40a2a28f70",
["resources"][0]["entity"]["credentials"]["issuerId"]: "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

GOT IT !
'''
x = 1

In [73]:
### Log In

import getpass
from subprocess import check_output
import json

print("Log in")
finished = False
while not finished:
    subprocess.call(["cf", "logout"])
    u = raw_input("username:")
    p = getpass.getpass("password:")
    proc = subprocess.Popen(["cf", "login", "-s", "dev", "-u", u, "-p", p],
                        shell=False,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
    stdout_value, stderr_value = proc.communicate()
    #print '\tstdout: ', repr(stdout_value)
    #print '\tstderr: ', repr(stderr_value)
    #print '\tproc.returncode: ', proc.returncode
    if proc.returncode == 0:
        finished = True
    else:
        print "login failed"

# squirrel away URLs for the space, apps and services
temp_p = json.loads(check_output(["cf", "curl", "/v2/spaces"]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "dev"):
        MY_DEV_SPACE_URL = line["metadata"]["url"]
        MY_APPS_URL      = line["entity"]["apps_url"]
        MY_SERVICES_URL  = line["entity"]["service_instances_url"]
        
print ("MY_DEV_SPACE_URL: " + MY_DEV_SPACE_URL)
print ("MY_APPS_URL: " +  MY_APPS_URL)
print ("MY_SERVICES_URL: " +  MY_SERVICES_URL)
print "done"

Log in
username:cwinsor@gmail.com
password:········
MY_DEV_SPACE_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc
MY_APPS_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
MY_SERVICES_URL: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
done


In [74]:
# Create UAA Service
# 3> cf create-service predix-uaa Tiered my-uaa -c '{"adminClientSecret":"my-secret"}'

from subprocess import check_output

print "Creating UAA Service..."
secret = getpass.getpass('secret code:')
aa = "\"adminClientSecret\""
bb = "\"" + secret + "\""
cc = "{" + aa + ":" + bb + "}"
#print cc
dd = ["cf", "create-service", "predix-uaa", "Free", "my-uaa", "-c", cc]
#print dd
foo = check_output(dd)
print foo

print "done"

Creating UAA Service...
secret code:········
Creating service instance my-uaa in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

done


In [75]:
# Get uaa vitals
# We need to get the environment variables of our new UAA.
# One way to accomplish that is to bind to a dummy application.
# Here we create a simple empty html file to serve as our dummy app.

import subprocess
import json

print "Get UAA vitals (will take a few seconds...)"
# create a temporary application
subprocess.check_call(["echo", "", ">", "cwinsor.temp.html"])
subprocess.check_call(["cf", "push", "--no-start", "cwinsor.temp.html"])
subprocess.check_call(["cf", "bs", "cwinsor.temp.html", "my-uaa"])
subprocess.check_call(["cf", "env", "cwinsor.temp.html"])

# we now will use that application to get environment variables relating to UAA

# find the UAA service GUID
# starting from the services URL, and search for my-uaa
# then from that URL get the GUID
temp_p = json.loads(check_output(["cf", "curl", MY_SERVICES_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "my-uaa"):
        MY_UAA_SERVICE_URL = line["metadata"]["url"]
print ("MY_UAA_SERVICE_URL:" + MY_UAA_SERVICE_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_UAA_SERVICE_URL]))
MY_UAA_SERVICE_GUID = temp_p["metadata"]["guid"]
print ("MY_UAA_SERVICE_GUID: " + MY_UAA_SERVICE_GUID)

# find the app
temp_p = json.loads(check_output(["cf", "curl", MY_APPS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_1_URL = line["metadata"]["url"]
#print ("MY_APP_1_URL:" + MY_APP_1_URL)

# find the issuerId of the service instance matching the UAA service
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))
MY_APP_1_SERVICE_BINDINGS_URL = temp_p["entity"]["service_bindings_url"]
#print ("MY_APP_1_SERVICE_BINDINGS_URL: " + MY_APP_1_SERVICE_BINDINGS_URL)

temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_SERVICE_BINDINGS_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["service_instance_guid"] == MY_UAA_SERVICE_GUID):
        MY_UAA_CREDENTIALS_ISSUER_ID = line["entity"]["credentials"]["issuerId"]
print ("MY_UAA_CREDENTIALS_ISSUER_ID: " + MY_UAA_CREDENTIALS_ISSUER_ID)

# delete the app
subprocess.check_call(["cf", "delete", "cwinsor.temp.html", "-f"])

print "done"

Get UAA vitals (will take a few seconds...)
MY_UAA_SERVICE_URL:/v2/service_instances/cfbcf94f-08f4-427b-a822-4a8d41a2d765
MY_UAA_SERVICE_GUID: cfbcf94f-08f4-427b-a822-4a8d41a2d765
MY_UAA_CREDENTIALS_ISSUER_ID: https://cfbcf94f-08f4-427b-a822-4a8d41a2d765.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token
done


In [76]:
# Create Asset Service
# 10> cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

from subprocess import check_output

print "Creating Asset Service..."
issuer_id = json.dumps({"trustedIssuerIds": [MY_UAA_CREDENTIALS_ISSUER_ID]})
foo = check_output(["cf", "create-service", "predix-asset", "Tiered", "my-asset", "-c", issuer_id])

print "done"

Creating Asset Service...
done


In [77]:
# Get Asset vitals
# We need to get the environment variables of our Asset.
# One way to accomplish that is to bind to a dummy application.
# Here we create a simple empty html file to serve as our dummy app.

import subprocess
import json

print "Get Asset vitals (will take a few seconds...)"
# create a temporary application
subprocess.check_call(["echo", "", ">", "cwinsor.temp.html"])
subprocess.check_call(["cf", "push", "--no-start", "cwinsor.temp.html"])
subprocess.check_call(["cf", "bs", "cwinsor.temp.html", "my-asset"])
subprocess.check_call(["cf", "env", "cwinsor.temp.html"])

# squirrel away the URL for the Asset Service
temp_p = json.loads(check_output(["cf", "curl", MY_SERVICES_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "my-asset"):
        MY_ASSET_SERVICE_URL = line["metadata"]["url"]
print ("MY_ASSET_SERVICE_URL: " + MY_ASSET_SERVICE_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_ASSET_SERVICE_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

MY_ASSET_SERVICE_GUID = temp_p["metadata"]["guid"]
print ("MY_ASSET_SERVICE_GUID: " + MY_ASSET_SERVICE_GUID)


# find the app
temp_p = json.loads(check_output(["cf", "curl", MY_APPS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_1_URL = line["metadata"]["url"]
print ("MY_APP_1_URL:" + MY_APP_1_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

MY_APP_1_SERVICE_BINDINGS_URL = temp_p["entity"]["service_bindings_url"]
print ("MY_APP_1_SERVICE_BINDINGS_URL: " + MY_APP_1_SERVICE_BINDINGS_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_SERVICE_BINDINGS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["credentials"]["instanceId"] == MY_ASSET_SERVICE_GUID):
        MY_ASSET_SERVICE_URI = line["entity"]["credentials"]["uri"]
print ("MY_ASSET_SERVICE_URI: " + MY_ASSET_SERVICE_URI)
print "done"

Get Asset vitals (will take a few seconds...)
MY_ASSET_SERVICE_URL: /v2/service_instances/62aadccd-1c4f-43e7-96c7-963b91171056
MY_ASSET_SERVICE_GUID: 62aadccd-1c4f-43e7-96c7-963b91171056
MY_APP_1_URL:/v2/apps/10e1e079-c26f-4286-8001-f7959fd5f108
MY_APP_1_SERVICE_BINDINGS_URL: /v2/apps/10e1e079-c26f-4286-8001-f7959fd5f108/service_bindings
MY_ASSET_SERVICE_URI: https://predix-asset.run.aws-usw02-pr.ice.predix.io
done


In [78]:
# UAAC Targets UAA

import subprocess
import json
import getpass

print "UAAC Targets UAA"
uaa_credentials_issuer = MY_UAA_CREDENTIALS_ISSUER_ID.split("/oauth/token")
subprocess.check_call(["uaac", "target", uaa_credentials_issuer[0]])

my_secret = getpass.getpass("UAAC secret:")
subprocess.check_call(["uaac", "token", "client", "get", "admin", "--secret", my_secret])

print "done"
    

UAAC Targets UAA
UAAC secret:········
done


In [79]:
# Add UAAC Client
# We need to use the instance id of Asset, sometimes called the Predix zone id

# 17> uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --secret my-secret --name asset-client

import subprocess
import json
import getpass

print "Add UAAC Client"
my_secret = getpass.getpass("UAAC secret:")
instanceid = "predix-asset.zones." + MY_ASSET_SERVICE_GUID + ".user"
l = ["uaac",
     "client",
     "add",
     "asset-client",
     "--authorities",
     "openid,uaa.none,uaa.resource," + instanceid,
     "--scope",
     "uaa.none,openid," + instanceid,
     "--autoapprove",
     "openid",
     "--authorized_grant_types",
     "authorization_code,client_credentials,refresh_token,password",
     "--secret",
     my_secret,
     "--name",
     "asset-client"]

subprocess.check_call(l)
print "done"

Add UAAC Client
UAAC secret:········
done


In [105]:
# Get Then Display The Token
import subprocess

#18> uaac token client get asset-client --secret my-secret # if token expires then do this again
#19> uaac context

print "Get Then Display The Token"
my_secret = getpass.getpass("UAAC secret:")

subprocess.check_call(["uaac", "token", "client", "get", "asset-client", "--secret", my_secret])
foo = check_output(["uaac", "context"])
#print(json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': ')))
#print foo
bar1 = foo.split("access_token: ")
bar2 = bar1[1].split("\n")
MY_BEARER_ACCESS_TOKEN = bar2[0]
print ("MY_BEARER_ACCESS_TOKEN: " + MY_BEARER_ACCESS_TOKEN)
print "done"


Get Then Display The Token
UAAC secret:········
MY_BEARER_ACCESS_TOKEN: eyJhbGciOiJSUzI1NiIsImtpZCI6ImxlZ2FjeS10b2tlbi1rZXkiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiI1OWVmOWYwNGJlNjg0MmE1ODMyZjJiYWQ3YTg0YWI5YSIsInN1YiI6ImFzc2V0LWNsaWVudCIsInNjb3BlIjpbInVhYS5yZXNvdXJjZSIsInByZWRpeC1hc3NldC56b25lcy42MmFhZGNjZC0xYzRmLTQzZTctOTZjNy05NjNiOTExNzEwNTYudXNlciIsIm9wZW5pZCIsInVhYS5ub25lIl0sImNsaWVudF9pZCI6ImFzc2V0LWNsaWVudCIsImNpZCI6ImFzc2V0LWNsaWVudCIsImF6cCI6ImFzc2V0LWNsaWVudCIsImdyYW50X3R5cGUiOiJjbGllbnRfY3JlZGVudGlhbHMiLCJyZXZfc2lnIjoiZWY5ZjIwOGEiLCJpYXQiOjE0ODA4NzEzMzQsImV4cCI6MTQ4MDkxNDUzNCwiaXNzIjoiaHR0cHM6Ly9jZmJjZjk0Zi0wOGY0LTQyN2ItYTgyMi00YThkNDFhMmQ3NjUucHJlZGl4LXVhYS5ydW4uYXdzLXVzdzAyLXByLmljZS5wcmVkaXguaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJjZmJjZjk0Zi0wOGY0LTQyN2ItYTgyMi00YThkNDFhMmQ3NjUiLCJhdWQiOlsicHJlZGl4LWFzc2V0LnpvbmVzLjYyYWFkY2NkLTFjNGYtNDNlNy05NmM3LTk2M2I5MTE3MTA1NiIsInVhYSIsImFzc2V0LWNsaWVudCIsIm9wZW5pZCJdfQ.Zwq1s3DZNRKQWEgv8EpcY8XXEknzoNIX-3emC_7JyqZAuJ9fkyRTVziCf8S_M9_XUPPRfO31bUbvL68QFCt_Du

In [112]:
# Validation By Talking To Asset
print "Validation By Talking To Asset"

#foo3 = ["curl",
#        "-H", [predix-zone-id: " + MY_ASSET_SERVICE_GUID + """,
#       "-H", "\"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\"",
#        MY_ASSET_SERVICE_URI]
#bar = check_output(foo3)
#print bar

#print "command 1"
THE_CMD  = " curl "
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


#curl -X POST -d '[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]'
#curl -X POST -d '[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]' 
#-H "Content-Type: application/json"
#-H "predix-zone-id: 87af97a0-c4a9-49fd-a002-240aacb3c540"
#-H "Authorization: <BEARER-TOKEN>"
#https://predix-asset.run.aws-usw02-pr.ice.predix.io/tests

#print "command 2"
THE_CMD  = ' curl -X POST -d \'[{ "uri": "/tests/my-first-test", "foo": true, "bar": [34,56] }]\' '
CONT_HDR = " -H \"Content-Type: application/json\" "
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI + "/tests "

proc = subprocess.Popen(THE_CMD + CONT_HDR + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


# curl -H "predix-zone-id: 87af97a0-c4a9-49fd-a002-240aacb3c540" -H "Authorization: <BEARER-TOKEN>" https://predix-asset.run.aws-usw02-pr.ice.predix.io/tests/my-first-test
#print "command 3"
THE_CMD  = ' curl '
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI + "/tests/my-first-test "

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)


# curl -H "predix-zone-id: 5aacd7de-f6da-42f6-8e3f-6c6c6dd595fd" -H "Authorization: <BEARER-TOKEN>" https://predix-asset.run.aws-usw02-pr.ice.predix.io 

#print "command 4"
THE_CMD  = ' curl '
ZONE_HDR = " -H \"predix-zone-id: " + MY_ASSET_SERVICE_GUID + "\" "
AUTH_HDR = " -H \"Authorization: bearer " + MY_BEARER_ACCESS_TOKEN + "\" "
THE_URI  = MY_ASSET_SERVICE_URI

proc = subprocess.Popen(THE_CMD + ZONE_HDR + AUTH_HDR + THE_URI,
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate()
print '\tstdout:', repr(stdout_value)
#print '\tstderr:', repr(stderr_value)

print "done"

Validation By Talking To Asset
	stdout: '[{"collection":"tests","count":1}]'
	stdout: ''
	stdout: '[\n  {\n    "uri": "/tests/my-first-test",\n    "foo": true,\n    "bar": [\n      34,\n      56\n    ]\n  }\n]'
	stdout: '[{"collection":"tests","count":1}]'
done


In [34]:

#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [5]:
### Log In

import getpass
from subprocess import check_output

print("cloudForge log in")
u = raw_input("username:")
p = getpass.getpass("password:")

# get the API endpoint
print ("get the API endpoint...")
foo = check_output(["cf", "login", "-u", u, "-p", p])
foo = check_output(["cf", "api"])
bar = foo.split()
MY_PR_API_ENDPOINT = bar[2]
print ("MY_PR_API_ENDPOINT " + MY_PR_API_ENDPOINT)

# get the OAUTH bearer token
print ("get the OAUTH bearer token...")
foo = check_output(["cf", "oauth-token"])
bar = foo.split()
MY_PR_BEARER_TOKEN = bar[1]
MY_PR_BEARER_TOKEN_W_AUTH = "'Authorization: bearer " + MY_PR_BEARER_TOKEN + "'"
print ("MY_PR_BEARER_TOKEN " + MY_PR_BEARER_TOKEN)

print("done")

cloudForge log in
username:cwinsor@gmail.com
password:········
get the API endpoint...
MY_PR_API_ENDPOINT https://api.system.aws-usw02-pr.ice.predix.io
get the OAUTH bearer token...
MY_PR_BEARER_TOKEN eyJhbGciOiJSUzI1NiIsImtpZCI6ImxlZ2FjeS10b2tlbi1rZXkiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiIwZjRlY2VjNTBmNTI0Y2QxODkzZmMxYTI3NDkwMTg2OSIsInN1YiI6IjM1NjNmOTE2LThiOWYtNDRhMi1iMTAzLTFiY2RiNmE1ZjJkNiIsInNjb3BlIjpbImNsb3VkX2NvbnRyb2xsZXIucmVhZCIsInBhc3N3b3JkLndyaXRlIiwiY2xvdWRfY29udHJvbGxlci53cml0ZSIsIm9wZW5pZCIsInVhYS51c2VyIl0sImNsaWVudF9pZCI6ImNmIiwiY2lkIjoiY2YiLCJhenAiOiJjZiIsImdyYW50X3R5cGUiOiJwYXNzd29yZCIsInVzZXJfaWQiOiIzNTYzZjkxNi04YjlmLTQ0YTItYjEwMy0xYmNkYjZhNWYyZDYiLCJvcmlnaW4iOiJ1YWEiLCJ1c2VyX25hbWUiOiJjd2luc29yQGdtYWlsLmNvbSIsImVtYWlsIjoiY3dpbnNvckBnbWFpbC5jb20iLCJyZXZfc2lnIjoiNjMyMzE2ZSIsImlhdCI6MTQ4MDYwMzc5MywiZXhwIjoxNDgwNjEwOTkzLCJpc3MiOiJodHRwczovL3VhYS5zeXN0ZW0uYXdzLXVzdzAyLXByLmljZS5wcmVkaXguaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJ1YWEiLCJhdWQiOlsiY2xvdWRfY29udHJvbGxlciIsInBhc3N3b3JkIiwiY2YiLCJ1

In [40]:
# Get Asset vitals
# We need to get the environment variables of our Asset.
# One way to accomplish that is to bind to a dummy application.
# Here we create a simple empty html file to serve as our dummy app.

import subprocess
import json

print "Get Asset vitals (will take a few seconds...)"
# create a temporary application
subprocess.check_call(["echo", "", ">", "cwinsor.temp.html"])
subprocess.check_call(["cf", "push", "--no-start", "cwinsor.temp.html"])
subprocess.check_call(["cf", "bs", "cwinsor.temp.html", "my-asset"])
subprocess.check_call(["cf", "env", "cwinsor.temp.html"])

# squirrel away the URL for the Asset Service
temp_p = json.loads(check_output(["cf", "curl", MY_SERVICES_URL]))
for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "my-asset"):
        MY_ASSET_SERVICE_URL = line["metadata"]["url"]
print ("MY_ASSET_SERVICE_URL: " + MY_ASSET_SERVICE_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_ASSET_SERVICE_URL]))
MY_ASSET_SERVICE_GUID = temp_p["metadata"]["guid"]
print ("MY_ASSET_SERVICE_GUID: " + MY_ASSET_SERVICE_GUID)


# find the app
temp_p = json.loads(check_output(["cf", "curl", MY_APPS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_1_URL = line["metadata"]["url"]
print ("MY_APP_1_URL:" + MY_APP_1_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

MY_APP_1_SERVICE_BINDINGS_URL = temp_p["entity"]["service_bindings_url"]
print ("MY_APP_1_SERVICE_BINDINGS_URL: " + MY_APP_1_SERVICE_BINDINGS_URL)
temp_p = json.loads(check_output(["cf", "curl", MY_APP_1_SERVICE_BINDINGS_URL]))
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["resources"]):
    if (line["entity"]["credentials"]["instanceId"] == MY_ASSET_SERVICE_GUID):
        MY_ASSET_SERVICE_URI = line["entity"]["credentials"]["uri"]
print ("MY_ASSET_SERVICE_URI: " + MY_ASSET_SERVICE_URI)

## delete the app
#subprocess.check_call(["cf", "delete", "cwinsor.temp.html", "-f"])

print "done"


Get Asset vitals (will take a few seconds...)
MY_ASSET_SERVICE_URL: /v2/service_instances/df8da196-d497-4f0e-9842-8e525f9d8ad0
MY_ASSET_SERVICE_GUID: df8da196-d497-4f0e-9842-8e525f9d8ad0
MY_APP_1_URL:/v2/apps/2e31abcd-835d-44c7-82e9-963463392b32
MY_APP_1_SERVICE_BINDINGS_URL: /v2/apps/2e31abcd-835d-44c7-82e9-963463392b32/service_bindings
{
    "next_url": null,
    "prev_url": null,
    "resources": [
        {
            "entity": {
                "app_guid": "2e31abcd-835d-44c7-82e9-963463392b32",
                "app_url": "/v2/apps/2e31abcd-835d-44c7-82e9-963463392b32",
                "binding_options": {},
                "credentials": {
                    "instanceId": "df8da196-d497-4f0e-9842-8e525f9d8ad0",
                    "scriptEngine_uri": "https://df8da196-d497-4f0e-9842-8e525f9d8ad0.predix-script-engine.run.aws-usw02-pr.ice.predix.io",
                    "uri": "https://predix-asset.run.aws-usw02-pr.ice.predix.io",
                    "zone": {
                   

In [6]:
# Identify what space to target.  Get the guid

from subprocess import check_output

# cf curl /v2/spaces
foo = check_output(["cf", "curl", "/v2/spaces"])
import json
bar = json.loads(foo)
print json.dumps(bar, sort_keys=True, indent=4, separators=(',', ': '))

dict = {}
for num, line in enumerate(bar["resources"]):
    dict[num] = line["entity"]["name"]

if (len(dict)==1):
    sp = 0;
else:
    finished = False
    while not finished:
        sp = input("Choose the space you want to target:\n     " + str(dict))
        try:
            t = bar["resources"][sp]
            finished = True
        except IndexError:
            finished = False

MY_PR_GUID   = bar["resources"][sp]["metadata"]["guid"]
MY_PR_URL    = bar["resources"][sp]["metadata"]["url"]
MY_PR_NAME   = bar["resources"][sp]["entity"]["name"]
MY_PR_ENTITY = bar["resources"][sp]["entity"]

lst = ["cf", "target", "-s", MY_PR_NAME]
foo = check_output(lst)

print "targeting space: \"" + MY_PR_NAME + "\""
print "url: " + MY_PR_URL

{
    "next_url": null,
    "prev_url": null,
    "resources": [
        {
            "entity": {
                "allow_ssh": true,
                "app_events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/app_events",
                "apps_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
                "auditors_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/auditors",
                "developers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/developers",
                "domains_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/domains",
                "events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/events",
                "isolation_segment_guid": null,
                "managers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/managers",
                "name": "dev",
                "organization_guid": "bd806ab3-1297-47cd-bdcc-1ab816ae2216",
                "organization_url": "/v2/organizations/bd806ab3-1297-47c

In [7]:
### Create UAA Service
### cf create-service predix-uaa Free my-uaa -c '{"adminClientSecret":"my-secret"}'

from subprocess import check_output

print("Create UAA Service")
secret = getpass.getpass('secret code:')
aa = "\"adminClientSecret\""
bb = "\"" + secret + "\""
cc = "{" + aa + ":" + bb + "}"
#print cc
dd = ["cf", "create-service", "predix-uaa", "Free", "my-uaa", "-c", cc]
#print dd
foo = check_output(dd)
print foo

# now get the details of the UAA and print GUID
lst = ["curl", "-H", "Authorization: bearer " + MY_PR_BEARER_TOKEN,  MY_PR_API_ENDPOINT + MY_PR_URL + "/summary"]
temp_j = check_output(lst)
import json
temp_p = json.loads(temp_j)
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["services"]):
    if (line["name"] == "my-uaa"):
        MY_PR_UAA = line

MY_PR_UAA_GUID   =  MY_PR_UAA["guid"]

print json.dumps(MY_PR_UAA, sort_keys=True, indent=4, separators=(',', ': '))
print "UAA_GUID: " + MY_PR_UAA_GUID
print "done"


Create UAA Service
secret code:········
Creating service instance my-uaa in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

{
    "bound_app_count": 0,
    "dashboard_url": null,
    "guid": "5b5428fa-4922-4be6-81ed-ad40a2a28f70",
    "last_operation": {
        "created_at": "2016-12-01T14:50:16Z",
        "description": "",
        "state": "succeeded",
        "type": "create",
        "updated_at": null
    },
    "name": "my-uaa",
    "service_plan": {
        "guid": "7b3e7f66-b1b8-448e-857e-79e2fde2e6c8",
        "name": "Free",
        "service": {
            "guid": "a7d692d5-5d26-4b0b-b9a9-8e3e84a61856",
            "label": "predix-uaa",
            "provider": null,
            "version": null
        }
    }
}
UAA_GUID: 5b5428fa-4922-4be6-81ed-ad40a2a28f70
done


In [15]:
### Create application bound to UAA Service (to get necessary parameters)

#foo = check_output(["echo", "", ">", "cwinsor.temp.html"])
#foo = check_output(["cf", "push", "--no-start", "cwinsor.temp.html"])
#foo = check_output(["cf", "bs", "cwinsor.temp.html", "my-uaa"])
#foo = check_output(["cf", "env", "cwinsor.temp.html"])

# we now will use that application to get environment variables relating to UAA

# navigate starting at the top
my_top = json.loads(check_output(["cf", "curl", "/v2/spaces"]))
#print json.dumps(my_top, sort_keys=True, indent=4, separators=(',', ': '))
# look for "dev" space
for num, line in enumerate(my_top["resources"]):
    if (line["entity"]["name"] == "dev"):
        MY_SPACE_URL = line["metadata"]["url"]
        break
print MY_SPACE_URL

# look at that space
my_space = json.loads(check_output(["cf", "curl", MY_SPACE_URL]))
print json.dumps(my_space, sort_keys=True, indent=4, separators=(',', ': '))

# look for the application

print "done"

/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc
{
    "entity": {
        "allow_ssh": true,
        "app_events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/app_events",
        "apps_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
        "auditors_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/auditors",
        "developers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/developers",
        "domains_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/domains",
        "events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/events",
        "isolation_segment_guid": null,
        "managers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/managers",
        "name": "dev",
        "organization_guid": "bd806ab3-1297-47cd-bdcc-1ab816ae2216",
        "organization_url": "/v2/organizations/bd806ab3-1297-47cd-bdcc-1ab816ae2216",
        "routes_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/routes",
        "security_groups_ur

In [55]:
### Create Asset Service
### cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

print("Create Asset Service")
#issuer_id = json.dumps({"trustedIssuerIds": [MY_PR_API_ENDPOINT + MY_PR_URL + "/oauth/token"]})
#issuer_id = json.dumps({"trustedIssuerIds": [MY_PR_API_ENDPOINT + MY_PR_URL + "/oauth/token"]})
#https://api.system.aws-usw02-pr.ice.predix.io/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/oauth/token"]}
foo = "https://f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"
issuer_id = json.dumps({"trustedIssuerIds": [foo]})
issuer_id = "https://" + UAA_GUID +  UAA_LABEL + predix-uaa


#print issuer_id
dd = ["cf", "create-service", "predix-asset", "Tiered", "my-asset", "-c", issuer_id]
foo = check_output(dd)
print foo

# now get the details of the UAA and print GUID
lst = ["curl", "-H", "Authorization: bearer " + MY_PR_BEARER_TOKEN,  MY_PR_API_ENDPOINT + MY_PR_URL + "/summary"]
temp_j = check_output(lst)
import json
temp_p = json.loads(temp_j)
#print json.dumps(temp_p, sort_keys=True, indent=4, separators=(',', ': '))

for num, line in enumerate(temp_p["services"]):
    if (line["name"] == "my-asset"):
        MY_PR_ASSET = line

MY_PR_ASSET_GUID   =  MY_PR_ASSET["guid"]

print json.dumps(MY_PR_ASSET, sort_keys=True, indent=4, separators=(',', ': '))
print "ASSET_GUID: " + MY_PR_ASSET_GUID
print "done"

        

Create Asset Service
Creating service instance my-asset in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

Attention: The plan `Tiered` of service `predix-asset` is not free.  The instance `my-asset` will incur a cost.  Contact your administrator if you think this is in error.


{
    "bound_app_count": 0,
    "dashboard_url": null,
    "guid": "43d66656-d604-45c8-9cad-91550a546e0f",
    "last_operation": {
        "created_at": "2016-12-01T00:34:01Z",
        "description": "",
        "state": "succeeded",
        "type": "create",
        "updated_at": null
    },
    "name": "my-asset",
    "service_plan": {
        "guid": "e08d80b5-6da3-4f29-a2f1-f7834d06cd29",
        "name": "Tiered",
        "service": {
            "guid": "90fc045c-adb0-41c6-9c5d-072d3d2c44b2",
            "label": "predix-asset",
            "provider": null,
            "version": null
        }
    }
}
ASSET_GUID: 43d66656-d604-45c8-9cad-91550a546e0f
done


In [49]:
# Set the UAA target for UAAC
# Fetch a token into the UAA
#15> uaac target https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io
#16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset

issuer_id = json.dumps({"trustedIssuerIds": [MY_PR_API_ENDPOINT + MY_PR_URL + "/oauth/token"]})
print issuer_id


#print  MY_PR_API_ENDPOINT + MY_PR_URL
#lst = ["uaac", "target",   MY_PR_API_ENDPOINT + MY_PR_URL]
#foo = check_output(lst)
#bar = json.loads(foo)
#print json.dumps(bar, sort_keys=True, indent=4, separators=(',', ': '))




{"trustedIssuerIds": ["https://api.system.aws-usw02-pr.ice.predix.io/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/oauth/token"]}


In [18]:
import subprocess

print '\npopen3:'
proc = subprocess.Popen('cat -; echo "to stderr" 1>&2',
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        )
stdout_value, stderr_value = proc.communicate('through stdin to stdout')
print '\tpass through:', repr(stdout_value)
print '\tstderr      :', repr(stderr_value)


popen3:
	pass through: 'through stdin to stdout'
	stderr      : 'to stderr\n'


In [53]:
import subprocess

proc = subprocess.Popen(["touch", "temp02.txt"],
                        stdin=subprocess.PIPE,
                                                                    )
#proc.communicate("foo")
#stdout_value, stderr_value = proc.communicate()
#print 'stdout:', stdout_value
#print 'stderr:', stderr_value
print "done"


done
